In [34]:
import pandas as pd
import numpy as np
from numpy.core.records import fromarrays
from scipy.io import savemat

/tmp/ipykernel_33511/495685066.py:3: DeprecationWarning: numpy.core.records is deprecated and has been renamed to numpy._core.records. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.records.fromarrays.
  from numpy.core.records import fromarrays


In [35]:
choice = 0  #start from 0
devices = ['pfet_g5v0d10v5']

# widths used for characterization and fringe cap parameters (fringe caps are not included in ngspice output)
w = np.array([5])
nfing = np.array([1])
cgdo_w = np.array([1.94170991e-10])*1e-6

In [36]:
def round_to_nearest(x, multiple=0.05):
    return (np.round(x / multiple) * multiple).astype(float)


In [49]:
# read ngspice data
df_raw = pd.read_csv('./simulation/techsweep_'+devices[choice]+'.txt', sep='\s+')
par_names = df_raw.columns.to_list()
fet_name = par_names[1].split('[')[0]

# remove unwanted columns and rename for readability
df = df_raw.drop(['frequency', 'frequency.1'], axis=1)
df = df.apply(pd.to_numeric)
df.columns = df.columns.str.replace(fet_name, '')
df.columns = df.columns.str.replace(fet_name[1:], '')
df.columns = df.columns.str.replace('onoise..', 'n')
df.columns = df.columns.str.removeprefix('[')
df.columns = df.columns.str.removesuffix(']')

# round sweep vectors to easily addressable values
df['l'] = df['l'].apply(lambda x: round(x/1e-6, 3))
df['vgs'] = df['vgs'].apply(lambda x: round_to_nearest(x))
df['vds'] = df['vds'].apply(lambda x: round_to_nearest(x))
df['vbs'] = df['vbs'].apply(lambda x: round_to_nearest(x,0.2))
df['vbs'] = df['vbs'].replace(-0.6000000000000001, -0.4)
df

,capbd,capbs,cdd,cgb,cgd,cgg,cgs,css,gds,gm,gmbs,id,l,vbs,vds,vgs,vth,n1overf,nid
0,2.368000e-15,2.368000e-15,9.501000e-17,-2.365000e-15,-8.124000e-17,2.395000e-15,5.119000e-17,-2.306000e-17,6.599000e-14,3.222000e-38,-7.098000e-32,0.000000e+00,0.5,0.0,0.00,0.00,0.9364,0.000000e+00,2.496000e-19
1,2.214000e-15,2.214000e-15,5.403000e-17,-1.981000e-15,-4.771000e-17,2.000000e-15,2.890000e-17,-1.340000e-17,7.733000e-15,4.654000e-40,-8.269000e-33,0.000000e+00,0.5,-0.2,-0.00,0.00,0.9925,0.000000e+00,2.925000e-20
2,2.094000e-15,2.094000e-15,3.235000e-17,-1.738000e-15,-2.935000e-17,1.750000e-15,1.734000e-17,-8.104000e-18,1.102000e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.5,-0.4,0.00,0.00,1.0430,0.000000e+00,4.167000e-21
3,2.326000e-15,2.368000e-15,4.602000e-18,-2.368000e-15,-3.502000e-18,2.390000e-15,-1.854000e-17,1.840000e-17,9.377000e-15,3.454000e-14,1.774000e-14,1.584000e-15,0.5,0.0,0.05,-0.00,0.9357,3.870000e-19,1.315000e-19
4,2.182000e-15,2.214000e-15,2.568000e-18,-1.983000e-15,-2.001000e-18,1.997000e-15,-1.220000e-17,1.027000e-17,1.166000e-15,4.331000e-15,1.913000e-15,1.897000e-16,0.5,-0.2,0.05,0.00,0.9918,4.741000e-20,1.589000e-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397834,1.200000e-15,2.197000e-15,5.486000e-18,2.554000e-15,-4.844000e-18,3.193000e-14,-3.448000e-14,2.187000e-14,2.919000e-06,1.280000e-04,4.308000e-05,2.991000e-04,3.0,-0.2,4.90,4.95,1.0790,2.646000e-09,1.034000e-12
397835,1.185000e-15,2.080000e-15,5.453000e-18,1.949000e-15,-4.907000e-18,3.174000e-14,-3.368000e-14,2.147000e-14,2.821000e-06,1.261000e-04,4.033000e-05,2.909000e-04,3.0,-0.4,4.90,4.95,1.1390,2.591000e-09,1.017000e-12
397836,1.212000e-15,2.344000e-15,5.122000e-18,3.529000e-15,-4.415000e-18,3.217000e-14,-3.570000e-14,2.240000e-14,2.957000e-06,1.302000e-04,4.612000e-05,3.080000e-04,3.0,-0.0,4.95,4.95,1.0140,2.706000e-09,1.052000e-12
397837,1.196000e-15,2.196000e-15,5.149000e-18,2.554000e-15,-4.546000e-18,3.193000e-14,-3.448000e-14,2.187000e-14,2.858000e-06,1.281000e-04,4.310000e-05,2.992000e-04,3.0,-0.2,4.95,4.95,1.0790,2.649000e-09,1.034000e-12


In [50]:
df['vbs'].min()

np.float64(-0.4)

In [51]:
# sweep variable vectors
l =   np.unique(df['l'])
vgs = np.unique(df['vgs'])
vds = np.unique(df['vds'])
vsb = np.unique(-df['vbs'])

In [53]:
# ngspice sweep order is l, vgs, vds, vsb
dims = [len(l), len(vgs), len(vds), len(vsb)]


# Verificar tamanhos das listas
print("Tamanho de l:", len(l))
print("Tamanho de vgs:", len(vgs))
print("Tamanho de vds:", len(vds))
print("Tamanho de vsb:", len(vsb))

# Verificar número de elementos no DataFrame
print("Número de elementos em df['id'].values:", df['id'].values.size)

# Calcular produto das dimensões
produto_dims = np.prod(dims)
print("Produto das dimensões:", produto_dims)

# Comparar produto das dimensões com número de elementos
if produto_dims != df['id'].values.size:
    print("Erro: O produto das dimensões não corresponde ao número de elementos no DataFrame")
else:
    print("Dimensões corretas")




id = np.reshape(df['id'].values, dims)
vt = np.reshape(df['vth'].values, dims)
gm = np.reshape(df['gm'].values, dims)
gmb = np.reshape(df['gmbs'].values, dims)
gds = np.reshape(df['gds'].values, dims)
cfringe = w[choice]*cgdo_w[choice]
cgg = np.reshape(df['cgg'].values, dims) \
      + 2*cfringe
cgb = -np.reshape(df['cgb'].values, dims)
cgd = -np.reshape(df['cgd'].values, dims) \
      + cfringe
cgs = -np.reshape(df['cgs'].values, dims) \
      + cfringe
cdd = np.reshape(df['cdd'].values, dims) \
      + np.reshape(df['capbd'].values, dims) \
      + cfringe
css = np.reshape(df['css'].values, dims) \
      + np.reshape(df['capbd'].values, dims) \
      + cfringe
sth = np.reshape(df['nid'].values, dims)**2
sfl = np.reshape(df['n1overf'].values, dims)**2


Tamanho de l: 13
Tamanho de vgs: 101
Tamanho de vds: 101
Tamanho de vsb: 3
Número de elementos em df['id'].values: 397839
Produto das dimensões: 397839
Dimensões corretas


In [54]:
dic = {
  "INFO": "SkyWater, 130nm CMOS, BSIM4",
  "CORNER": "NOM",
  "TEMP": 300.0,
  "VGS": vgs,
  "VDS": vds,
  "VSB": vsb,
  "L": l,
  "W": w[choice],
  "NFING": nfing[choice],
  "ID": id,
  "VT": vt,
  "GM": gm,
  "GMB": gmb,
  "GDS": gds,
  "CGG": cgg,
  "CGB": cgb,
  "CGD": cgd,
  "CGS": cgs,
  "CDD": cdd,
  "CSS": css,
  "STH": sth,
  "SFL": sfl
}
savemat('./simulation/'+devices[choice]+'.mat', {devices[choice]: dic})